In [1]:
import sys
sys.path.append('/home/amine.mcharrak/clear2025/dglearn/')

from dglearn import *

# example graph structure with 6 variables
n_vars = 6
edges = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 1), (5, 4)]
var_names = {i:"$X_%d$"%(i+1) for i in range(n_vars)}

# generate random parameters for true graph structure
B_support = edges2array(edges, n_vars) - np.eye(n_vars)
B_sampled, s_sampled = sample_param_unif(B_support, B_low=0.2, B_high=0.8, var_low=1.0, var_high=3.0)

# generate samples from distribution specified by sampled parameters
X = sample_graph(B_sampled, s_sampled, 10000)

# learn structure using tabu search, plot learned structure
tabu_length = 4
patience = 4

manager = CyclicManager(X, bic_coef=0.5)
learned_support, best_score, log = tabu_search(manager, tabu_length, patience, first_ascent=False, verbose=1) # returns a binary matrix as learned support

# perform virtual edge correction
print("virtual edge correction...")
learned_support = virtual_refine(manager, learned_support, patience=0, max_path_len=6, verbose=1) # still is still a binary matrix

# remove any reducible edges
learned_support = reduce_support(learned_support, fill_diagonal=False)

# plot true and learned directed graphs
plot_collection({'true': edges, 'learned': array2edges(learned_support)}, n_vars, name_list=var_names,
                latex=False, n_cols=2, node_size=600, font_size=14, width=2, save_path="./assets/dg6_learning_result.png")

# compute performance metric: SHD
true_graph = AdjacencyStucture(n_vars, edge_list=edges)
search = GraphEquivalenceSearch(true_graph)
search.search_dfs()
equiv_class = [binary2array(bstr) for bstr in search.visited_graphs]

Add edge: 4 -> 1, Delta Score: 2.756e-01 best score: 0.2756288209166386
Add edge: 4 -> 3, Delta Score: 1.382e-01 best score: 0.41380665739756883
Add edge: 2 -> 3, Delta Score: 1.119e-01 best score: 0.5256608592198844
Add edge: 0 -> 1, Delta Score: 6.950e-02 best score: 0.5951566659050848
Add edge: 5 -> 4, Delta Score: 4.961e-02 best score: 0.6447658421718796
Add edge: 2 -> 1, Delta Score: 3.090e-02 best score: 0.6756610363954064
Add edge: 3 -> 5, Delta Score: 2.199e-02 best score: 0.6976492192673613
Add edge: 3 -> 4, Delta Score: 6.061e-03 best score: 0.7037106966902625
Add edge: 1 -> 2, Delta Score: 3.447e-03 best score: 0.7071574260042943
Del edge: 4 -> 3, Delta Score: 4.539e-04 best score: 0.7076113629276689
Del edge: 2 -> 1, Delta Score: 3.933e-04 best score: 0.7080046210652564
Del edge: 3 -> 5, Delta Score: 4.358e-04 best score: 0.7084404089285486
Add edge: 0 -> 4, Delta Score: -3.731e-04 best score: 0.7084404089285486
Add edge: 1 -> 0, Delta Score: -4.181e-04 best score: 0.708440